In [ ]:
import numpy as np
import h5py

hf_x = h5py.File('../data/mfcc_x.h5', 'r')
X_train = hf_x.get('x_train_data')[()]

hf_y = h5py.File('../data/mfcc_y.h55', 'r')
y_train = hf_y.get('y_train_data')[()]

(X_train.shape, y_train.shape)

In [ ]:
from tensorflow.keras.layers import GRU, GlobalMaxPooling2D, Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, Conv3D, ConvLSTM2D
from tensorflow.keras.models import Sequential, Model
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import train_test_split

gru = Sequential()
#model.add(GRU(256,input_shape=(99,26)))
gru.add(GRU(256,input_shape=(49,13)))
gru.add(Dropout(0.5))
gru.add(Dense(12, activation='softmax'))
#model.add(Dense(12, activation='sigmoid'))
gru.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
gru.summary()
weights = gru.get_weights()

In [ ]:
NUM_FOLDS = 4
EPOCHS = 10
BATCH_SIZE = 16
BAGS = 5

kf = GroupKFold(n_splits=NUM_FOLDS)

kf = GroupKFold(n_splits=NUM_FOLDS)

for bag in range(BAGS):
    fold = 0

    val_loss = np.ones((EPOCHS,NUM_FOLDS),np.float32)

    for train, val in kf.split(x_train,y_train,G):
        gru.set_weights(weights)
        gru.reset_states()
        history = gru.fit(x_train[train], y_train[train], batch_size=BATCH_SIZE, validation_data=(x_train[val], y_train[val]), epochs=EPOCHS, shuffle=True, verbose=1)
        val_loss[:,fold] = history.history['val_loss']
        fold += 1

    val_mean = np.mean(val_loss,axis=1)
    best_loss = np.min(val_mean)
    best_epoch = np.argmin(val_mean)
    print('Best epoch: {} Best loss: {}'.format(best_epoch,best_loss))
    
    gru.set_weights(weights)
    gru.reset_states()
    gru.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=best_epoch, shuffle=True, verbose=1)

    #model.save('models/gru_{}_{}.h5'.format(bag+1,best_loss))